In [5]:
import torchaudio
import numpy as np
train_dataset = torchaudio.datasets.VoxCeleb1Identification('/mnt/d/VoxCeleb1Identification/data', subset='train', download=False)
test_dataset = torchaudio.datasets.VoxCeleb1Identification('/mnt/d/VoxCeleb1Identification/data', subset='test', download=False)

In [6]:


# present_train_audio_files = []
# missing_train_audio_files = []
# sub_train_len = len(train_dataset) #4000
# for i in range(sub_train_len):
#   try:
#     train_dataset[i][2]
#     present_train_audio_files.append(i)
#   except:
#     missing_train_audio_files.append(i)

# present_test_audio_files = []
# missing_test_audio_files = []
# sub_test_len = len(test_dataset) #200
# for i in range(sub_test_len):
#   try:
#     test_dataset[i][2]
#     present_test_audio_files.append(i)
#   except:
#     missing_test_audio_files.append(i) 

In [7]:
import json
# json.dump(present_train_audio_files, open("present_train_audio_files.json", 'w'))
# json.dump(present_test_audio_files, open("present_test_audio_files.json", 'w'))
present_train_audio_files = json.load(open("present_train_audio_files.json", 'r'))
present_test_audio_files = json.load(open("present_test_audio_files.json", 'r'))

There are 4361 missing files in train dataset and 279 missing files in test dataset. Lets remove those samples.

In [8]:
from torch.utils.data import Subset
from torch.utils.data import Dataset
import torch

class VoxCeleb1IdentificationUnified(Dataset):
    
    def __init__(self, voxceleb1_dataset, present_audio_files = []):
        self.voxceleb1_dataset = voxceleb1_dataset
        self.voxceleb1_dataset = Subset(self.voxceleb1_dataset, present_audio_files)
        
        self.num_samples_per_clip = 80000   
        self.mel_spectrogram_transformation = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,
            n_fft=1024,
            hop_length=512,
            n_mels=64)

    def __getitem__(self, idx):
        waveform, _, target, _ = self.voxceleb1_dataset[idx]
        waveform = self._right_zero_pad(self._cut_if_necessary(waveform))
        mel_spec = self.mel_spectrogram_transformation(waveform)
        return mel_spec, target
    
    def __len__(self):
        return len(self.voxceleb1_dataset)
    
    def _right_zero_pad(self, signal):
      length_signal = signal.shape[1]
      if length_signal < self.num_samples_per_clip:
          num_missing_samples = self.num_samples_per_clip - length_signal
          signal = torch.nn.functional.pad(signal, (0, num_missing_samples))
      return signal
    
    def _cut_if_necessary(self, signal):
      if signal.shape[1] > self.num_samples_per_clip:
          signal = signal[:, :self.num_samples_per_clip]
      return signal
  
train_subset = VoxCeleb1IdentificationUnified(train_dataset, present_train_audio_files[:1000])
test_subset = VoxCeleb1IdentificationUnified(train_dataset, present_test_audio_files[:100])

In [9]:
idx = np.random.randint(0, 100)
print(idx)
print(f'train: {train_subset[idx]}')
print(f'test: {test_subset[idx]}')

99
train: (tensor([[[6.0013e+00, 8.2566e-01, 1.0710e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.5570e+00, 5.8077e-01, 5.7693e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.6105e+01, 2.3266e+00, 2.9071e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [9.4109e-04, 1.6834e-04, 1.2673e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.8099e-04, 1.5889e-04, 1.3632e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [7.0682e-04, 8.5380e-05, 7.2833e-05,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]]), 2)
test: (tensor([[[6.0013e+00, 8.2566e-01, 1.0710e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.5570e+00, 5.8077e-01, 5.7693e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.6105e+01, 2.3266e+00, 2.9071e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [9.4109e-04, 1.6834e-04, 1.2673e-04,  ..., 0.000

In [10]:
print(f'Length of train dataset: {len(train_subset)}')
print(train_subset[0])
print(' ')
print(f'Length of test dataset: {len(test_subset)}')
print(test_subset[0])
speakers_test = []
speakers_train = []
# 1754
for i in range(len(test_subset)):
  speakers_test.append(test_subset[i][1])
print(' ')
print(f'There are {len(set(speakers_test))} unique speakers in first {len(test_subset)} samples of test set')

for i in range(len(train_subset)):
  speakers_train.append(train_subset[i][1])
print(' ')
print(f'There are {len(set(speakers_train))} unique speakers in first {len(train_subset)} samples of train set')

Length of train dataset: 1000
(tensor([[[1.3433, 0.0434, 0.2335,  ..., 0.0924, 0.2549, 1.7636],
         [1.5431, 0.1776, 2.0001,  ..., 0.1316, 0.2775, 1.9115],
         [1.8138, 0.2381, 1.3203,  ..., 0.5002, 0.4647, 2.3463],
         ...,
         [0.1240, 0.5055, 0.1610,  ..., 0.6842, 0.5754, 0.7417],
         [0.1975, 0.3604, 0.1840,  ..., 0.4042, 0.5048, 0.4415],
         [0.0902, 0.3216, 0.0680,  ..., 0.0362, 0.0680, 0.0960]]]), 1)
 
Length of test dataset: 100
(tensor([[[1.3433, 0.0434, 0.2335,  ..., 0.0924, 0.2549, 1.7636],
         [1.5431, 0.1776, 2.0001,  ..., 0.1316, 0.2775, 1.9115],
         [1.8138, 0.2381, 1.3203,  ..., 0.5002, 0.4647, 2.3463],
         ...,
         [0.1240, 0.5055, 0.1610,  ..., 0.6842, 0.5754, 0.7417],
         [0.1975, 0.3604, 0.1840,  ..., 0.4042, 0.5048, 0.4415],
         [0.0902, 0.3216, 0.0680,  ..., 0.0362, 0.0680, 0.0960]]]), 1)


 
There are 2 unique speakers in first 100 samples of test set
 
There are 12 unique speakers in first 1000 samples of train set


In [11]:
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

test_speakers_classes = [str(i) for i in range(len(set(speakers_test)))]

# Prepare dataset classes

In [16]:
import numpy as np
from PIL import Image

from torch.utils.data.sampler import BatchSampler


class TripletVoxCeleb1ID(Dataset):
    """
    Train: For each sample (anchor) randomly chooses a positive and negative samples
    Test: Creates fixed triplets for testing
    """

    def __init__(self, voxceleb1_dataset, train=True):
        self.voxceleb1_dataset = voxceleb1_dataset
        self.train = train
        self.mel_spectrogram_transformation = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,
            n_fft=1024,
            hop_length=512,
            n_mels=64)
        self.num_samples_per_clip = 80000 # 5 sec

        if self.train:
            self.train_labels = torch.tensor([self.voxceleb1_dataset[i][1]
                                              for i in range(len(self.voxceleb1_dataset))])
            
            self.labels_set = set(self.train_labels.numpy())
            self.label_to_indices = {label: np.where(self.train_labels.numpy() == label)[0]
                                     for label in self.labels_set}

        else:
            self.test_labels = torch.tensor([self.voxceleb1_dataset[i][1] 
                                              for i in range(len(self.voxceleb1_dataset))])
        
            # generate fixed triplets for testing
            self.labels_set = set(self.test_labels.numpy())
            self.label_to_indices = {label: np.where(self.test_labels.numpy() == label)[0]
                                     for label in self.labels_set}

            random_state = np.random.RandomState(29)

            triplets = [[i,
                         random_state.choice(self.label_to_indices[self.test_labels[i].item()]),
                         random_state.choice(self.label_to_indices[
                                                 np.random.choice(
                                                     list(self.labels_set - set([self.test_labels[i].item()]))
                                                 )
                                             ])
                         ]
                        for i in range(len(self.test_labels))]
            self.test_triplets = triplets

    def __getitem__(self, index):
        if self.train:
            # audio1, label1 = self._right_zero_pad(self._cut_if_necessary(self.voxceleb1_dataset[index][0])), self.train_labels[index].item()
            audio1, label1 = self.voxceleb1_dataset[index][0], self.train_labels[index].item()
            positive_index = index
            while positive_index == index:
                positive_index = np.random.choice(self.label_to_indices[label1])
                #! 99% that for one particular speaker there is just 1 recording 
                #TODO Create histogram of number of samples assigned to one speaker
            negative_label = np.random.choice(list(self.labels_set - set([label1])))
            negative_index = np.random.choice(self.label_to_indices[negative_label])
            # audio2 = self._right_zero_pad(self._cut_if_necessary(self.voxceleb1_dataset[positive_index][0]))
            # audio3 = self._right_zero_pad(self._cut_if_necessary(self.voxceleb1_dataset[negative_index][0]))
            audio2 = self.voxceleb1_dataset[positive_index][0]
            audio3 = self.voxceleb1_dataset[negative_index][0]
        else:
            # audio1 = self._right_zero_pad(self._cut_if_necessary(self.voxceleb1_dataset[self.test_triplets[index][0]][0]))
            # audio2 = self._right_zero_pad(self._cut_if_necessary(self.voxceleb1_dataset[self.test_triplets[index][1]][0]))
            # audio3 = self._right_zero_pad(self._cut_if_necessary(self.voxceleb1_dataset[self.test_triplets[index][2]][0]))
            audio1 = self.voxceleb1_dataset[self.test_triplets[index][0]][0]
            audio2 = self.voxceleb1_dataset[self.test_triplets[index][1]][0]
            audio3 = self.voxceleb1_dataset[self.test_triplets[index][2]][0]
                                          
        # spec1 = self.mel_spectrogram_transformation(audio1)
        # spec2 = self.mel_spectrogram_transformation(audio2)
        # spec3 = self.mel_spectrogram_transformation(audio3)   
        spec1 = audio1
        spec2 = audio2
        spec3 = audio3 

        return (spec1, spec2, spec3), []
    
    # def _right_zero_pad(self, signal):
    #   length_signal = signal.shape[1]
    #   if length_signal < self.num_samples_per_clip:
    #       num_missing_samples = self.num_samples_per_clip - length_signal
    #       signal = torch.nn.functional.pad(signal, (0, num_missing_samples))
    #   return signal
    
    # def _cut_if_necessary(self, signal):
    #   if signal.shape[1] > self.num_samples_per_clip:
    #       signal = signal[:, :self.num_samples_per_clip]
    #   return signal

    def __len__(self):
        return len(self.voxceleb1_dataset)

In [17]:
triplet_dataset = TripletVoxCeleb1ID(train_subset, train=True)
print(triplet_dataset.label_to_indices)
triplet_dataset = TripletVoxCeleb1ID(test_subset, train=False)
print(triplet_dataset.label_to_indices)

{1: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56]), 2: array([ 57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174]), 3: array([175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187

In [18]:
triplet_dataset[0][0][0].shape

torch.Size([1, 64, 157])

# Neural Networks

In [19]:
import torch.nn as nn
import torch.nn.functional as F


class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(nn.Conv2d(1, 32, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(64, 128, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2))

        self.fc = nn.Sequential(nn.Linear(8192, 128), # 6*4*4x256
                                nn.PReLU(),
                                nn.Linear(128, 256),
                                nn.PReLU(),
                                nn.Linear(256, 128)
                                )

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)

class EmbeddingNetL2(EmbeddingNet):
    def __init__(self):
        super(EmbeddingNetL2, self).__init__()

    def forward(self, x):
        output = super(EmbeddingNetL2, self).forward(x)
        output /= output.pow(2).sum(1, keepdim=True).sqrt()
        return output

    def get_embedding(self, x):
        return self.forward(x)
        
class SiameseNet(nn.Module):
    def __init__(self, embedding_net):
        super(SiameseNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        return output1, output2

    def get_embedding(self, x):
        return self.embedding_net(x)


class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        output3 = self.embedding_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

In [20]:
from torchsummary import summary
model = EmbeddingNet()
summary(model, (1, 64, 157))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 60, 153]             832
             PReLU-2          [-1, 32, 60, 153]               1
         MaxPool2d-3           [-1, 32, 30, 76]               0
            Conv2d-4           [-1, 64, 26, 72]          51,264
             PReLU-5           [-1, 64, 26, 72]               1
         MaxPool2d-6           [-1, 64, 13, 36]               0
            Conv2d-7           [-1, 128, 9, 32]         204,928
             PReLU-8           [-1, 128, 9, 32]               1
         MaxPool2d-9           [-1, 128, 4, 16]               0
           Linear-10                  [-1, 128]       1,048,704
            PReLU-11                  [-1, 128]               1
           Linear-12                  [-1, 256]          33,024
            PReLU-13                  [-1, 256]               1
           Linear-14                  [

# Training

In [21]:
import torch
import numpy as np
from tqdm import tqdm


def fit(train_loader, val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[],
        start_epoch=0):
    """
    Loaders, model, loss function and metrics should work together for a given task,
    i.e. The model should be able to process data output of loaders,
    loss function should process target output of loaders and outputs from the model

    Examples: Classification: batch loader, classification model, NLL loss, accuracy metric
    Siamese network: Siamese loader, siamese model, contrastive loss
    Online triplet learning: batch loader, embedding model, online triplet loss
    """
    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):
        scheduler.step()
        pbar = tqdm(total=1)
        # Train stage
        train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)

        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        val_loss, metrics = test_epoch(val_loader, model, loss_fn, cuda, metrics)
        val_loss /= len(val_loader)

        message += '\nEpoch: {}/{}. Validation set: Average loss: {:.4f}'.format(epoch + 1, n_epochs,
                                                                                 val_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())
        
        # update progress bar
        pbar.update(1)
        pbar.set_description(f"Epoch {epoch+1}/{n_epochs} train_loss={train_loss:.2f} val_loss={val_loss:.2f}")
        pbar.close()

        # print(message)


def train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics):
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    batch_idx = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        target = target if len(target) > 0 else None
        if not type(data) in (tuple, list):
            data = (data,)
        if cuda:
            data = tuple(d.cuda() for d in data)
            if target is not None:
                target = target.cuda()


        optimizer.zero_grad()
        outputs = model(*data)

        if type(outputs) not in (tuple, list):
            outputs = (outputs,)

        loss_inputs = outputs
        if target is not None:
            target = (target,)
            loss_inputs += target

        loss_outputs = loss_fn(*loss_inputs)
        loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
        losses.append(loss.item())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for metric in metrics:
            metric(outputs, target, loss_outputs)

        if batch_idx % log_interval == 0:
            message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data[0]), len(train_loader.dataset), # type: ignore
                #! if batch size is smaller than defined then it may become smaller.
                100. * batch_idx / len(train_loader), np.mean(losses))
            for metric in metrics:
                message += '\t{}: {}'.format(metric.name(), metric.value())

            print(message)
            losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics


def test_epoch(val_loader, model, loss_fn, cuda, metrics):
    with torch.no_grad():
        for metric in metrics:
            metric.reset()
        model.eval()
        val_loss = 0
        for batch_idx, (data, target) in enumerate(val_loader):
            target = target if len(target) > 0 else None
            if not type(data) in (tuple, list):
                data = (data,)
            if cuda:
                data = tuple(d.cuda() for d in data)
                if target is not None:
                    target = target.cuda()

            outputs = model(*data)

            if type(outputs) not in (tuple, list):
                outputs = (outputs,)
            loss_inputs = outputs
            if target is not None:
                target = (target,)
                loss_inputs += target

            loss_outputs = loss_fn(*loss_inputs)
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            val_loss += loss.item()

            for metric in metrics:
                metric(outputs, target, loss_outputs)

    return val_loss, metrics

# Loss functions

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ContrastiveLoss(nn.Module):
    """
    Contrastive loss
    Takes embeddings of two samples and a target label == 1 if samples are from the same class and label == 0 otherwise
    """

    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(1)  # squared distances
        losses = 0.5 * (target.float() * distances +
                        (1 + -1 * target).float() * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2))
        return losses.mean() if size_average else losses.sum()


class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()

# Main

In [23]:
triplet_train_dataset = TripletVoxCeleb1ID(train_subset) # Returns triplets of images
triplet_test_dataset = TripletVoxCeleb1ID(test_subset)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, **kwargs) # type: ignore
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, **kwargs) # type: ignore

# Set up the network and training parameters
margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 1
log_interval = 1

# train_epoch(triplet_train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics=[])
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

/home/armak/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 0/1 [00:00<?, ?it/s]

Train: [0/1000 (0%)]	Loss: 2083.989746
Train: [128/1000 (14%)]	Loss: 4755.961426
Train: [256/1000 (29%)]	Loss: 1667.810547
Train: [384/1000 (43%)]	Loss: 178.662827
Train: [512/1000 (57%)]	Loss: 1822.876221
Train: [640/1000 (71%)]	Loss: 2275.473145
Train: [768/1000 (86%)]	Loss: 877.841431


ZeroDivisionError: division by zero

In [24]:
def plot_embeddings(embeddings, targets, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5)
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(test_speakers_classes)

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 128))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

In [25]:
train_subset_unified = VoxCeleb1IdentificationUnified(train_dataset, present_audio_files= present_train_audio_files)
train_loader = torch.utils.data.DataLoader(train_subset_unified, batch_size=batch_size, shuffle=True, drop_last=True, **kwargs) # type: ignore

test_subset_unified = VoxCeleb1IdentificationUnified(test_dataset, present_audio_files= present_test_audio_files)
test_loader = torch.utils.data.DataLoader(test_subset_unified, batch_size=batch_size, shuffle=False, drop_last=True, **kwargs) # type: ignore

train_embeddings_tl, train_labels_tl = extract_embeddings(train_loader, embedding_net)
plot_embeddings(train_embeddings_tl, train_labels_tl)
val_embeddings_tl, val_labels_tl = extract_embeddings(test_loader, embedding_net)
plot_embeddings(val_embeddings_tl, val_labels_tl)